<a href="https://colab.research.google.com/github/Yung-Huei/Python/blob/main/HW1_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/BIOS740/Homework/Homework 1
#!pip install pytorch_lightning
#!git clone https://github.com/RunpengDai/ICSA_DLcourse.git
#!git clone https://github.com/RunpengDai/BIOS740.git

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/BIOS740/Homework/Homework 1


In [8]:
import glob #讀取檔案路徑，例如找到所有.CSV檔
import pandas as pd #讀取表格資料（如 CSV）與 DataFrame 操作
import numpy as np #提供高效的陣列處理
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader #自訂數據集的類別/用來分批（batch）讀取資料，支援打亂、並行處理等功能
from sklearn.model_selection import train_test_split #用來將資料集依比例分成訓練集與測試集
import pytorch_lightning as pl #可以減少訓練時的 boilerplate code and 管理訓練流程、自動處理 GPU/TPU、記錄等。
from torchvision import models,transforms #圖像處理: 預訓練模型（如 ResNet、VGG）/資料預處理（如 resize、normalize、data augmentation）
import torchvision.transforms.functional as TF #例如隨機旋轉、裁切、水平翻轉
from pytorch_lightning.callbacks.early_stopping import EarlyStopping #提早停止訓練的回呼函數（Callback），如果驗證集的表現停止進步，就會自動停止訓練以避免 overfitting。
from pytorch_lightning.loggers import CSVLogger #將訓練過程的紀錄（如損失、準確率）寫入 .csv 檔，方便後續分析。
import matplotlib.pyplot as plt

**Predicting Alzheimer's Disease Status Using Hippocampal Data**

阿茲海默症（Alzheimer's Disease, AD）是一種神經退化性疾病，其特徵為認知功能逐漸衰退。海馬迴的萎縮（hippocampus atrophy）是阿茲海默症早期的生物標記之一，而海馬迴是對記憶形成非常關鍵的大腦區域。

*   Source: ADNI資料庫
*   Sample Size: 369 位受試者的左側與右側海馬迴矩陣 (extracted from MRI scans)


    1.   Left Hippocampus Matrix: Dimensions 7x150x100 per subject.
    2.   Right Hippocampus Matrix: Dimensions 7x150x100 per subject.
    3.   Feature Description: radial distance from the medial model; the (log)determinant and two eigenvalues of the Jacobian matrix from the surface tensor-basedmorphometry (TBM) model (referred to as the determinant, eigenvalue1, and eigenvalue2); and three features from the multivariate surface TBM (mTBM) model (referred to as the mTBM1, mTBM2, and mTBM3.
    4.   AD Status Labels: A separate CSV file indicating the AD status (AD or Control) for each subject.


*   Format: 壓縮的 CSV 檔案
*   Method: CNN



In [38]:
!ls
# # 複製 zip 檔案到 Colab 當前目錄
#!cp "/content/drive/MyDrive/BIOS740/Homework/Homework 1/HW1_data/LeftCSV.zip"
#!cp "/content/drive/MyDrive/BIOS740/Homework/Homework 1/HW1_data/RightCSV.zip"
#!ls

 adni_subset.csv   cnn-sol.ipynb      selected_samples_subset.txt
'Chap3 HW.docx'    HW1_data
 cnn.ipynb	   requirements.txt


In [42]:
# Data Loading:
# Decompress and load the CSV files containing the hippocampal matrices.
# https://drive.google.com/drive/folders/1X1HPde-5snvjZhUyYKu5BHtqoJyCLrYb?usp=share_link

# 解壓縮
%cd /content/drive/MyDrive/BIOS740/Homework/Homework 1/HW1_data
!unzip -o RightCSV.zip -d "/content/drive/MyDrive/BIOS740/Homework/Homework 1/"
!unzip -o LeftCSV.zip -d "/content/drive/MyDrive/BIOS740/Homework/Homework 1/"
!ls
print("=========")
%cd /content/drive/MyDrive/BIOS740/Homework/Homework 1/
subject_ids = np.loadtxt("selected_samples_subset.txt", dtype=str)
left_hippo_dir = "LeftCSV_subset/"
left_hippo_files = glob.glob(left_hippo_dir+"*")
right_hippo_dir = "RightCSV_subset/"
right_hippo_files = glob.glob(right_hippo_dir+"*")
labels = pd.read_csv("adni_subset.csv",names=['ID','AD'])
print(subject_ids[:5])
print(left_hippo_files[:3])

#Describe the dimensions and structure of the data.


/content/drive/.shortcut-targets-by-id/1X1HPde-5snvjZhUyYKu5BHtqoJyCLrYb/HW1_data
Archive:  RightCSV.zip
  inflating: /content/drive/MyDrive/BIOS740/Homework/Homework 1/RightCSV_subset/002_S_0295__2011-06-02__S110477.csv  
  inflating: /content/drive/MyDrive/BIOS740/Homework/Homework 1/RightCSV_subset/002_S_0413__2011-06-16__S111989.csv  
  inflating: /content/drive/MyDrive/BIOS740/Homework/Homework 1/RightCSV_subset/002_S_0685__2010-07-15__S89145.csv  
  inflating: /content/drive/MyDrive/BIOS740/Homework/Homework 1/RightCSV_subset/002_S_1261__2011-03-14__S101641.csv  
  inflating: /content/drive/MyDrive/BIOS740/Homework/Homework 1/RightCSV_subset/002_S_1280__2011-05-04__S108096.csv  
  inflating: /content/drive/MyDrive/BIOS740/Homework/Homework 1/RightCSV_subset/002_S_4213__2011-09-02__S121165.csv  
  inflating: /content/drive/MyDrive/BIOS740/Homework/Homework 1/RightCSV_subset/002_S_4225__2011-09-21__S122885.csv  
  inflating: /content/drive/MyDrive/BIOS740/Homework/Homework 1/RightC